In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.7.1


In [2]:
!python -V

Python 3.13.5


In [13]:
import pickle
import pandas as pd
import os

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/greg/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/greg/anaconda3/envs/exp-tracking-env/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
year = 2023
month = 3

df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1. What's the standard deviation of the predicted duration for this dataset?

In [11]:
std_duration = y_pred.std()
print(std_duration)

6.247488852238704


### Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output. 

First, let's create an artificial `ride_id` column:

```python
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results. 

Save it as parquet:

```python
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What's the size of the output file?

In [ ]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

output_file = 'output.parquet'

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

file_size_bytes = os.path.getsize(output_file)
file_size_mb = file_size_bytes / 1024 / 1024
print(file_size_mb)

59.49517822265625
